In [1]:
import pandas as pd
import re


In [2]:
cases = pd.read_csv("justice.csv")
print(cases.isna().sum())
cases.dropna(inplace=True)
print(cases.isna().sum())
cases.head()

Unnamed: 0              0
ID                      0
name                    0
href                    0
docket                  0
term                    0
first_party             1
second_party            1
facts                   0
facts_len               0
majority_vote           0
minority_vote           0
first_party_winner     15
decision_type           7
disposition            72
issue_area            142
dtype: int64
Unnamed: 0            0
ID                    0
name                  0
href                  0
docket                0
term                  0
first_party           0
second_party          0
facts                 0
facts_len             0
majority_vote         0
minority_vote         0
first_party_winner    0
decision_type         0
disposition           0
issue_area            0
dtype: int64


,Unnamed: 0,ID,name,href,docket,term,first_party,second_party,facts,facts_len,majority_vote,minority_vote,first_party_winner,decision_type,disposition,issue_area
1,1,50613,Stanley v. Illinois,https://api.oyez.org/cases/1971/70-5014,70-5014,1971,"Peter Stanley, Sr.",Illinois,<p>Joan Stanley had three children with Peter ...,757,5,2,True,majority opinion,reversed/remanded,Civil Rights
2,2,50623,Giglio v. United States,https://api.oyez.org/cases/1971/70-29,70-29,1971,John Giglio,United States,<p>John Giglio was convicted of passing forged...,495,7,0,True,majority opinion,reversed/remanded,Due Process
3,3,50632,Reed v. Reed,https://api.oyez.org/cases/1971/70-4,70-4,1971,Sally Reed,Cecil Reed,"<p>The Idaho Probate Code specified that ""male...",378,7,0,True,majority opinion,reversed/remanded,Civil Rights
4,4,50643,Miller v. California,https://api.oyez.org/cases/1971/70-73,70-73,1971,Marvin Miller,California,"<p>Miller, after conducting a mass mailing cam...",305,5,4,True,majority opinion,vacated/remanded,First Amendment
5,5,50644,Kleindienst v. Mandel,https://api.oyez.org/cases/1971/71-16,71-16,1971,"Richard G. Kleindienst, Attorney General of th...","Ernest E. Mandel, et al.",<p>Ernest E. Mandel was a Belgian professional...,2282,6,3,True,majority opinion,reversed,First Amendment


In [3]:
cases.drop(columns=['Unnamed: 0', 'ID', 'name', 'href', 'docket', 'term',  
                    'facts_len', 'majority_vote', 'minority_vote', 'decision_type', 'disposition', 'issue_area'], inplace=True)
cases.head()

,first_party,second_party,facts,first_party_winner
1,"Peter Stanley, Sr.",Illinois,<p>Joan Stanley had three children with Peter ...,True
2,John Giglio,United States,<p>John Giglio was convicted of passing forged...,True
3,Sally Reed,Cecil Reed,"<p>The Idaho Probate Code specified that ""male...",True
4,Marvin Miller,California,"<p>Miller, after conducting a mass mailing cam...",True
5,"Richard G. Kleindienst, Attorney General of th...","Ernest E. Mandel, et al.",<p>Ernest E. Mandel was a Belgian professional...,True


In [4]:
cases = cases.rename(columns={'first_party_winner': 'winning_party'})
cases

,first_party,second_party,facts,winning_party
1,"Peter Stanley, Sr.",Illinois,<p>Joan Stanley had three children with Peter ...,True
2,John Giglio,United States,<p>John Giglio was convicted of passing forged...,True
3,Sally Reed,Cecil Reed,"<p>The Idaho Probate Code specified that ""male...",True
4,Marvin Miller,California,"<p>Miller, after conducting a mass mailing cam...",True
5,"Richard G. Kleindienst, Attorney General of th...","Ernest E. Mandel, et al.",<p>Ernest E. Mandel was a Belgian professional...,True
...,...,...,...,...
3297,"Janet L. Yellen, Secretary of the Treasury",Confederated Tribes of the Chehalis Reservatio...,<p>For over a century after the Alaska Purchas...,True
3298,United States,Refugio Palomar-Santiago,"<p>Refugio Palomar-Santiago, a Mexican nationa...",True
3299,Tarahrick Terry,United States,<p>Tarahrick Terry pleaded guilty to one count...,False
3300,United States,Joshua James Cooley,<p>Joshua James Cooley was parked in his picku...,True


In [5]:
for i, row in cases.iterrows():
    if row['winning_party'] == True:
        row['winning_party'] = row['first_party']
    else:
        row['winning_party'] = row['second_party']
cases

,first_party,second_party,facts,winning_party
1,"Peter Stanley, Sr.",Illinois,<p>Joan Stanley had three children with Peter ...,"Peter Stanley, Sr."
2,John Giglio,United States,<p>John Giglio was convicted of passing forged...,John Giglio
3,Sally Reed,Cecil Reed,"<p>The Idaho Probate Code specified that ""male...",Sally Reed
4,Marvin Miller,California,"<p>Miller, after conducting a mass mailing cam...",Marvin Miller
5,"Richard G. Kleindienst, Attorney General of th...","Ernest E. Mandel, et al.",<p>Ernest E. Mandel was a Belgian professional...,"Richard G. Kleindienst, Attorney General of th..."
...,...,...,...,...
3297,"Janet L. Yellen, Secretary of the Treasury",Confederated Tribes of the Chehalis Reservatio...,<p>For over a century after the Alaska Purchas...,"Janet L. Yellen, Secretary of the Treasury"
3298,United States,Refugio Palomar-Santiago,"<p>Refugio Palomar-Santiago, a Mexican nationa...",United States
3299,Tarahrick Terry,United States,<p>Tarahrick Terry pleaded guilty to one count...,United States
3300,United States,Joshua James Cooley,<p>Joshua James Cooley was parked in his picku...,United States


In [7]:
cases['facts'] = cases['facts'].str.replace(r'<[^<]+?>', '', regex=True)
cases['facts'] = cases['facts'].str.lower().apply(lambda x: re.sub(r'[^a-z\s]', '', x)).apply(lambda x: ' '.join([word for word in x.split() if len(word) > 1]))
cases

,first_party,second_party,facts,winning_party
1,"Peter Stanley, Sr.",Illinois,joan stanley had three children with peter sta...,"Peter Stanley, Sr."
2,John Giglio,United States,john giglio was convicted of passing forged mo...,John Giglio
3,Sally Reed,Cecil Reed,the idaho probate code specified that males mu...,Sally Reed
4,Marvin Miller,California,miller after conducting mass mailing campaign ...,Marvin Miller
5,"Richard G. Kleindienst, Attorney General of th...","Ernest E. Mandel, et al.",ernest mandel was belgian professional journal...,"Richard G. Kleindienst, Attorney General of th..."
...,...,...,...,...
3297,"Janet L. Yellen, Secretary of the Treasury",Confederated Tribes of the Chehalis Reservatio...,for over century after the alaska purchase in ...,"Janet L. Yellen, Secretary of the Treasury"
3298,United States,Refugio Palomar-Santiago,refugio palomarsantiago mexican national was g...,United States
3299,Tarahrick Terry,United States,tarahrick terry pleaded guilty to one count of...,United States
3300,United States,Joshua James Cooley,joshua james cooley was parked in his pickup t...,United States


In [15]:
cases['facts'].iloc[4]

'ernest mandel was belgian professional journalist and editorinchief of la guache belgian left socialist weekly publication he described himself as revolutionary marxist advocating the economic governmental and international doctrines of world communism previously the united states twice allowed mandel to temporarily visit the united states once as working journalist in and once as lecturer in both times and without mandels knowledge the state department found him ineligible but the attorney general used his discretionary power under the immigration and nationality act of to admit mendel temporarily the graduate student association at stanford university invited mandel to the united states for six days to participate in conference on september mandel applied to the american counsel in brussels for nonimmigrant visa to enter the united states other persons invited mandel to additional events and mandel filed second visa application in october detailing more extensive itinerary on octobe

use bert for classification
use sep token to include parties in case facts

tokenizer.sep_token

df.first_party + " [SEP] " + df.second_party + " [sep] " + 

bert for sequence classification 

gpt-2 simple 